<a href="https://colab.research.google.com/github/scienclick/ScikitLearnPipeline4NLP/blob/master/PyTorchdatashape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:

import numpy as np
import pandas as pd
import nltk
import re
from torchtext.legacy import data
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
stop_words = nltk.corpus.stopwords.words("english")
from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import torch
import torch.nn as nn

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [19]:
def tokenize(text):
    '''this method does the following
    1. normalizing all the words to lower size
    2. removes punctuations
    3. splits the words
    4. removes the stopwords like am,is,have,you,...
    5. lammetizes the words for example running-->run
    '''
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())    # normalize case and remove punctuation
    tokens = word_tokenize(text)    # tokenize text
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]    # lemmatize andremove stop words
    return tokens

In [20]:
text=["tree is beautiful definietly 100%","seal likes fish too","nature is see and tree ", "Engineering is borring","Programming is more fun","life is such like 0 chanse"]
label=[0,0,0,1,1,2]
ddf=pd.DataFrame({"label":label,"text":text})
ddf

,label,text
0,0,tree is beautiful definietly 100%
1,0,seal likes fish too
2,0,nature is see and tree
3,1,Engineering is borring
4,1,Programming is more fun
5,2,life is such like 0 chanse


In [21]:
ddf.to_csv('thedata-explore.csv', index=False)

In [67]:
# TEXT = data.Field(sequential=True, batch_first=True, lower=True, pad_first=True, tokenize=tokenize)
TEXT = data.Field(sequential=True, batch_first=True, lower=True, pad_first=True,)

LABEL = data.Field(sequential=False, use_vocab=False, is_target=True)

dataset = data.TabularDataset(path='thedata-explore.csv', format='csv', skip_header=True,
                              fields=[('label', LABEL), ('text', TEXT)]
                              )

In [68]:
train_dataset, test_dataset = dataset.split(split_ratio=.75)  # default is 0.7
len(train_dataset.examples)

4

In [69]:
ex = dataset.examples[3]
print(ex.text)
print(ex.label)

['engineering', 'is', 'borring']
1


In [70]:
MAX_VOCAB_SIZE = 20

TEXT.build_vocab(train_dataset,
                 max_size=MAX_VOCAB_SIZE,
                 # vectors="glove.6B.100d.txt",
                 # unk_init=torch.Tensor.normal_,
                 min_freq=1,
                 )
LABEL.build_vocab(train_dataset)

In [71]:
for i in range(len(TEXT.vocab)):
    print(TEXT.vocab.itos[i])
print("---------------------------")
for i in range(len(LABEL.vocab)):
    print(LABEL.vocab.itos[i])

<unk>
<pad>
is
0
and
borring
chanse
engineering
fish
life
like
likes
nature
seal
see
such
too
tree
---------------------------
<unk>
0
1
2


In [72]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator,  test_iterator = data.BucketIterator.splits(
    (train_dataset,  test_dataset),
    batch_size=BATCH_SIZE,
    sort_key=lambda x: len(x.text),
    sort_within_batch=False,
    device=device)

In [73]:
for batch in train_iterator:
    # print("imput:", inputs.shape, "shape:", targets.shape)
    print("imput:", batch.text, "shape:", batch.label)


imput: tensor([[ 1,  1, 13, 11,  8, 16],
        [ 1, 12,  2, 14,  4, 17],
        [ 1,  1,  1,  7,  2,  5],
        [ 9,  2, 15, 10,  3,  6]]) shape: tensor([0, 0, 1, 2])


In [74]:
for i in batch.text:
    print("imput:", [TEXT.vocab.itos[j] for j in i])

imput: ['<pad>', '<pad>', 'seal', 'likes', 'fish', 'too']
imput: ['<pad>', 'nature', 'is', 'see', 'and', 'tree']
imput: ['<pad>', '<pad>', '<pad>', 'engineering', 'is', 'borring']
imput: ['life', 'is', 'such', 'like', '0', 'chanse']


In [77]:
for batch in train_iterator:
    print("imput:", batch.text, "shape:", batch.label)
    print("----------------------------------")
    for i in batch.text:
        print("imput:", [TEXT.vocab.itos[j] for j in i])

    em=nn.Embedding(len(TEXT.vocab), 3)(batch.text.cpu())
    print("*1---------------------------------")
    print(em)
    print("---------------------------------")
    print(em.shape)

imput: tensor([[ 1,  1,  1,  7,  2,  5],
        [ 9,  2, 15, 10,  3,  6],
        [ 1,  1, 13, 11,  8, 16],
        [ 1, 12,  2, 14,  4, 17]]) shape: tensor([1, 2, 0, 0])
----------------------------------
imput: ['<pad>', '<pad>', '<pad>', 'engineering', 'is', 'borring']
imput: ['life', 'is', 'such', 'like', '0', 'chanse']
imput: ['<pad>', '<pad>', 'seal', 'likes', 'fish', 'too']
imput: ['<pad>', 'nature', 'is', 'see', 'and', 'tree']
*1---------------------------------
tensor([[[ 0.3384,  0.5570, -1.1350],
         [ 0.3384,  0.5570, -1.1350],
         [ 0.3384,  0.5570, -1.1350],
         [ 0.6288, -0.1714, -0.2968],
         [-0.2097, -0.6370, -0.6590],
         [ 1.8518, -1.2153,  0.1500]],

        [[-0.7118, -1.1152, -0.3627],
         [-0.2097, -0.6370, -0.6590],
         [ 0.1636, -0.0309,  1.2614],
         [ 0.1731,  1.7366, -1.1559],
         [-0.4311, -0.3124, -1.4238],
         [-0.2652,  1.5784,  0.4129]],

        [[ 0.3384,  0.5570, -1.1350],
         [ 0.3384,  0.5570

In [79]:
for batch in train_iterator:
    print("imput:", batch.text, "shape:", batch.label)
    print("----------------------------------")
    for i in batch.text:
        print("imput:", [TEXT.vocab.itos[j] for j in i])

    em=nn.Embedding(len(TEXT.vocab), 3)(batch.text.cpu())
    em=em.permute(0, 2, 1)
    print("*2---------------------------------")
    print(em)
    print("---------------------------------")
    print(em.shape)

imput: tensor([[ 1,  1, 13, 11,  8, 16],
        [ 9,  2, 15, 10,  3,  6],
        [ 1, 12,  2, 14,  4, 17],
        [ 1,  1,  1,  7,  2,  5]]) shape: tensor([0, 2, 0, 1])
----------------------------------
imput: ['<pad>', '<pad>', 'seal', 'likes', 'fish', 'too']
imput: ['life', 'is', 'such', 'like', '0', 'chanse']
imput: ['<pad>', 'nature', 'is', 'see', 'and', 'tree']
imput: ['<pad>', '<pad>', '<pad>', 'engineering', 'is', 'borring']
*2---------------------------------
tensor([[[-0.1761, -0.1761, -0.2384,  0.2754, -0.2645, -0.3102],
         [ 0.9617,  0.9617, -0.2439,  1.1979, -2.1688, -1.1989],
         [-1.3849, -1.3849, -0.4108, -0.1680,  0.1365, -0.4480]],

        [[ 0.0313, -0.9375, -1.2891,  1.4668,  1.7250, -1.0237],
         [ 0.8918,  0.6981,  0.3701, -0.1802,  0.3027, -0.3452],
         [-0.7358, -0.5887,  0.9754, -0.1357, -0.6981,  1.1448]],

        [[-0.1761,  1.3165, -0.9375,  0.1613, -1.9093,  2.2364],
         [ 0.9617,  2.0524,  0.6981,  0.2597, -0.2617,  1.5429],


In [80]:
for batch in train_iterator:
    print("imput:", batch.text, "shape:", batch.label)
    print("----------------------------------")
    for i in batch.text:
        print("imput:", [TEXT.vocab.itos[j] for j in i])

    em=nn.Embedding(len(TEXT.vocab), 3)(batch.text.cpu())

    em=em.permute(0, 2, 1)

    c=nn.Conv1d(in_channels=3, out_channels=32, kernel_size=2, padding=1)(em)
    print("*3---------------------------------")
    print(c)
    print("----------------------------------")

    print(c.shape)

imput: tensor([[ 1,  1,  1,  7,  2,  5],
        [ 9,  2, 15, 10,  3,  6],
        [ 1, 12,  2, 14,  4, 17],
        [ 1,  1, 13, 11,  8, 16]]) shape: tensor([1, 2, 0, 0])
----------------------------------
imput: ['<pad>', '<pad>', '<pad>', 'engineering', 'is', 'borring']
imput: ['life', 'is', 'such', 'like', '0', 'chanse']
imput: ['<pad>', 'nature', 'is', 'see', 'and', 'tree']
imput: ['<pad>', '<pad>', 'seal', 'likes', 'fish', 'too']
*3---------------------------------
tensor([[[-5.3554e-01, -6.2198e-01, -6.2198e-01, -2.5777e-01, -1.2642e-01,
           2.7617e-01, -5.2339e-01],
         [-7.7039e-01, -7.8869e-01, -7.8869e-01, -6.9216e-01, -1.3907e+00,
           6.4337e-01,  1.6118e-03],
         [ 8.5423e-01,  1.0953e+00,  1.0953e+00,  9.0420e-01,  1.1357e+00,
          -2.1333e-01,  2.3084e-01],
         [-3.6586e-01,  6.8581e-01,  6.8581e-01,  7.1870e-01, -9.0954e-01,
           1.3028e+00, -7.6729e-01],
         [-3.6967e-01, -5.9134e-01, -5.9134e-01, -7.0309e-01, -3.7847e-01,
 

In [81]:
for batch in train_iterator:
    print("imput:", batch.text, "shape:", batch.label)
    print("---------------------------------")

    for i in batch.text:
        print("imput:", [TEXT.vocab.itos[j] for j in i])

    em=nn.Embedding(len(TEXT.vocab), 3)(batch.text.cpu())
    em=em.permute(0, 2, 1)

    c=nn.Conv1d(in_channels=3, out_channels=32, kernel_size=2, padding=1)(em)
    p=nn.MaxPool1d(2)(c)
    print("*3---------------------------------")

    print(p)
    print("---------------------------------")

    print(p.shape)

imput: tensor([[ 9,  2, 15, 10,  3,  6],
        [ 1,  1,  1,  7,  2,  5],
        [ 1,  1, 13, 11,  8, 16],
        [ 1, 12,  2, 14,  4, 17]]) shape: tensor([2, 1, 0, 0])
---------------------------------
imput: ['life', 'is', 'such', 'like', '0', 'chanse']
imput: ['<pad>', '<pad>', '<pad>', 'engineering', 'is', 'borring']
imput: ['<pad>', '<pad>', 'seal', 'likes', 'fish', 'too']
imput: ['<pad>', 'nature', 'is', 'see', 'and', 'tree']
*3---------------------------------
tensor([[[ 0.7630,  1.0241,  0.1259],
         [-0.2652,  0.2799, -0.6315],
         [-0.8898,  0.0705,  0.3301],
         [-0.4626, -0.6172,  0.7400],
         [ 0.3662,  0.4131,  0.3637],
         [ 1.0642,  0.9323, -0.3762],
         [ 0.2799,  0.5064,  0.5313],
         [ 0.6897,  0.1545,  0.1460],
         [ 1.0685,  0.5423, -0.1613],
         [ 0.7099,  0.3879, -0.1065],
         [ 0.3349, -0.0366,  0.3504],
         [ 0.0099, -0.2381,  0.7677],
         [-0.7351, -0.1716,  0.1261],
         [ 0.3708,  0.9690, -0.

In [82]:
for batch in train_iterator:
    print("imput:", batch.text, "shape:", batch.label)
    print("---------------------------------")

    for i in batch.text:
        print("imput:", [TEXT.vocab.itos[j] for j in i])

    em=nn.Embedding(len(TEXT.vocab), 3)(batch.text.cpu())

    em=em.permute(0, 2, 1)

    c=nn.Conv1d(in_channels=3, out_channels=32, kernel_size=2, padding=1)(em)

    p=nn.MaxPool1d(2)(c)
    print("*4---------------------------------")

    o = p.permute(0, 2, 1)
    print(o)
    print("---------------------------------")

    print(o.shape)

imput: tensor([[ 1,  1,  1,  7,  2,  5],
        [ 9,  2, 15, 10,  3,  6],
        [ 1,  1, 13, 11,  8, 16],
        [ 1, 12,  2, 14,  4, 17]]) shape: tensor([1, 2, 0, 0])
---------------------------------
imput: ['<pad>', '<pad>', '<pad>', 'engineering', 'is', 'borring']
imput: ['life', 'is', 'such', 'like', '0', 'chanse']
imput: ['<pad>', '<pad>', 'seal', 'likes', 'fish', 'too']
imput: ['<pad>', 'nature', 'is', 'see', 'and', 'tree']
*4---------------------------------
tensor([[[ 9.2426e-01,  4.9562e-01,  2.8078e-01, -3.7427e-01, -2.0895e-01,
           5.7905e-01,  2.7768e-01, -3.8824e-01,  1.8917e-01,  1.3327e-01,
           9.7212e-01, -9.8028e-02, -3.4746e-01,  5.9648e-02,  4.2253e-01,
           4.6274e-02,  3.9327e-01,  6.5917e-01,  3.4731e-01,  2.7528e-01,
           4.3690e-01, -8.8091e-02, -5.2617e-02,  1.9718e-01,  1.7459e-01,
          -5.4596e-01,  3.7665e-01,  1.6080e-01, -3.7634e-01, -1.5558e-01,
           4.9415e-01,  2.2199e-01],
         [ 9.2426e-01,  4.9562e-01, -9

In [84]:
 for batch in train_iterator:
    print("imput:", batch.text, "shape:", batch.label)
    print("---------------------------------")
    for i in batch.text:
        print("imput:", [TEXT.vocab.itos[j] for j in i])

    em=nn.Embedding(len(TEXT.vocab), 3)(batch.text.cpu())
    em=em.permute(0, 2, 1)
    c=nn.Conv1d(in_channels=3, out_channels=32, kernel_size=2, padding=1)(em)
    p=nn.MaxPool1d(2)(c)
    o = p.permute(0, 2, 1)
    print(o.shape)
    # max pool
    print("*5---------------------------------")
    o, _ = torch.max(o, 1)
    print(o)
    print("---------------------------------")
    print(o.shape)

imput: tensor([[ 1,  1, 13, 11,  8, 16],
        [ 9,  2, 15, 10,  3,  6],
        [ 1,  1,  1,  7,  2,  5],
        [ 1, 12,  2, 14,  4, 17]]) shape: tensor([0, 2, 1, 0])
---------------------------------
imput: ['<pad>', '<pad>', 'seal', 'likes', 'fish', 'too']
imput: ['life', 'is', 'such', 'like', '0', 'chanse']
imput: ['<pad>', '<pad>', '<pad>', 'engineering', 'is', 'borring']
imput: ['<pad>', 'nature', 'is', 'see', 'and', 'tree']
torch.Size([4, 3, 32])
*5---------------------------------
tensor([[ 6.7171e-01,  9.3287e-02,  4.4010e-01,  3.5686e-01,  1.2296e+00,
          6.3973e-01,  2.7642e-01,  6.4191e-01,  1.0323e+00,  1.3804e+00,
          6.1779e-01,  4.3577e-01,  1.0779e+00,  4.7280e-01,  6.0774e-01,
          3.3416e-01,  2.9088e-01,  2.5850e-01,  1.0748e+00,  3.3299e-01,
          7.9122e-01,  5.6351e-01,  7.4564e-01,  8.9999e-02,  1.2044e+00,
          1.6545e-01,  5.7618e-01,  4.4837e-01,  6.8160e-02,  6.3554e-01,
          2.1701e-01,  5.8270e-01],
        [ 6.5912e-01, 

In [85]:
for batch in train_iterator:
    print("imput:", batch.text, "shape:", batch.label)
    print("---------------------------------")

    for i in batch.text:
        print("imput:", [TEXT.vocab.itos[j] for j in i])

    em=nn.Embedding(len(TEXT.vocab), 3)(batch.text.cpu())

    em=em.permute(0, 2, 1)

    c=nn.Conv1d(in_channels=3, out_channels=32, kernel_size=2, padding=1)(em)

    p=nn.MaxPool1d(2)(c)

    o = p.permute(0, 2, 1)

    # max pool
    o, _ = torch.max(o, 1)

    l=nn.Linear(32, 3)(o)
    print("*6---------------------------------")

    print(l)
    print("---------------------------------")

    print(l.shape)

imput: tensor([[ 9,  2, 15, 10,  3,  6],
        [ 1,  1, 13, 11,  8, 16],
        [ 1, 12,  2, 14,  4, 17],
        [ 1,  1,  1,  7,  2,  5]]) shape: tensor([2, 0, 0, 1])
---------------------------------
imput: ['life', 'is', 'such', 'like', '0', 'chanse']
imput: ['<pad>', '<pad>', 'seal', 'likes', 'fish', 'too']
imput: ['<pad>', 'nature', 'is', 'see', 'and', 'tree']
imput: ['<pad>', '<pad>', '<pad>', 'engineering', 'is', 'borring']
*6---------------------------------
tensor([[-0.6624, -0.2285,  1.8863],
        [-0.7063,  0.0516,  1.3766],
        [-0.2594,  0.4202,  1.3718],
        [-0.6481,  0.1403,  1.2128]], grad_fn=<AddmmBackward>)
---------------------------------
torch.Size([4, 3])


This will be followed by
nn.CrossEntropyLoss() which combines LogSoftmax and NLLLoss. in other words it will output 4x3 matrix